In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mistral_finetuned/pytorch/default/1/kaggle/working/alpensicher_model_mistral-finetuned/adapter_model.safetensors
/kaggle/input/mistral_finetuned/pytorch/default/1/kaggle/working/alpensicher_model_mistral-finetuned/adapter_config.json
/kaggle/input/mistral_finetuned/pytorch/default/1/kaggle/working/alpensicher_model_mistral-finetuned/README.md
/kaggle/input/mistral_finetuned/pytorch/default/1/kaggle/working/alpensicher_model_mistral-finetuned/tokenizer.json
/kaggle/input/mistral_finetuned/pytorch/default/1/kaggle/working/alpensicher_model_mistral-finetuned/tokenizer_config.json
/kaggle/input/mistral_finetuned/pytorch/default/1/kaggle/working/alpensicher_model_mistral-finetuned/special_tokens_map.json
/kaggle/input/mistral_finetuned/pytorch/default/1/kaggle/working/alpensicher_model_mistral-finetuned/tokenizer.model


In [2]:
!pip install -q transformers[torch] datasets
!pip install -q bitsandbytes peft
!pip install trl==0.12.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 MB 25.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.8/374.8 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.5/450.5 kB 28.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 78.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 86.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.44.2
    Uninstalling transformers-4.44.2:
      Successfully uninstalled transformers-4.44.2


In [3]:
model = '/kaggle/input/mistral_finetuned/pytorch/default/1/kaggle/working/alpensicher_model_mistral-finetuned'

In [4]:
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer
model_hf = AutoPeftModelForCausalLM.from_pretrained(
    model,
    load_in_4bit = True,
)
tokenizer_hf = AutoTokenizer.from_pretrained(model)

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
/usr/local/lib/python3.10/dist-packages/transformers/quantizers/auto.py:186: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)
`low_cpu_mem_usage` was None, now default to True since model is quantized.


model.safetensors:   0%|          | 0.00/4.14G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/157 [00:00<?, ?B/s]

In [5]:
question = "Tell me all the different insurance products that are available for AlpenSicher Insurance AG?"

# Create the input prompt
prompt_test = f"<s>[INST] {question} [/INST]"

# Tokenize the input
inputs = tokenizer_hf(prompt_test, return_tensors="pt").to("cuda")

# Generate the response
outputs = model_hf.generate(**inputs, max_new_tokens=64, use_cache=True)

# Decode the response
response = tokenizer_hf.batch_decode(outputs, skip_special_tokens=False)[0]

# Clean up the response (remove any special tokens or prefixes)
response = response.split("[/INST]")[-1].strip()

print(f"Question: {question}")
print(f"Answer: {response}")

Question: Tell me all the different insurance products that are available for AlpenSicher Insurance AG?
Answer: The insurance products available for AlpenSicher Insurance AG include AlpenSicher Holiday Dream Package, AlpenSicher Business Guardian, AlpenSicher Health Elite, AlpenSicher Retirement Plus, and AlpenSicher Life Premium.</s>
